In [42]:
from __future__ import print_function
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


class ArrayCaster(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        print(data.shape)
        print(np.transpose(np.matrix(data)).shape)
        return np.transpose(np.matrix(data))

union =FeatureUnion([('comment', Pipeline([
            ('selector', ItemSelector(key='comment')),
            #('vect', CountVectorizer(ngram_range=(1,1), binary=True, min_df=3)),
            ('tfidf', TfidfVectorizer())
          ])
        ),

        ('compound', Pipeline([
            ('selector', ItemSelector(key='compound')),
            ('caster', ArrayCaster())
          ])
        )])

data_vader = pd.read_csv("train-balanced-sarcasm-vader.csv")


X= data_vader[["comment","compound"]]


X_train, X_test, y_train, y_test = train_test_split(X, data_vader["label"],test_size =0.3, random_state=50)
print(X_train.shape)
print(y_train.shape)

union.fit(X_train,y_train)
y = pipeline.predict(X_test)
print(report = classification_report(y_test, y, digits = 3))

(14000, 2)
(14000,)


ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 1, expected 6000.

In [21]:
from __future__ import print_function

import numpy as np
import pandas as pd


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split





class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]




pipeline = Pipeline([
    
    ('union', FeatureUnion(
        transformer_list=[


            # Pipeline for standard bag-of-words model for comments
            ('comment', Pipeline([
                ('selector', ItemSelector(key='comment')),
                ('tfidf', TfidfVectorizer()),
                
            ])),

            # Pipeline for pulling the compound value of the sentiment
            ('body_stats', Pipeline([
                ('selector', ItemSelector(key='compound')),
            ])),
        ],        
    )),

    # Use a SVC classifier on the combined features
    ('svc', SVC(kernel='linear')),
])


data_vader = pd.read_csv("train-balanced-sarcasm-vader.csv")


X= data_vader[["comment","compound"]]

X_train, X_test, y_train, y_test = train_test_split(X, data_vader["label"],test_size =0.3, random_state=50)
print(X_train.shape)
print(y_train.shape)

pipeline.fit(X_train,  y_train)
y = pipeline.predict(X_test)
print(report = classification_report(y_test, y, digits = 3))

(14000, 2)
(14000,)


ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 1, expected 14000.